In [1]:
import scanpy as sc
import os
import pandas as pd
import matplotlib as mpl
import sys
import numpy as np
import matplotlib.pyplot as plt
import diffxpy.api as de
import arviz as av

#sys.path.append('/wsfish/glioblastoma/')
#import FISHspace as sp
import pymc as pm

%reload_ext autoreload
%autoreload 2

mpl.rcParams['pdf.fonttype'] = 42

# save figure with no pad
mpl.rcParams['savefig.pad_inches'] = 0
mpl.rcParams['savefig.bbox'] = 'tight'

# set axes width
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['xtick.minor.pad'] = 2
mpl.rcParams['xtick.major.pad'] = 2
mpl.rcParams['ytick.minor.pad'] = 2
mpl.rcParams['ytick.major.pad'] = 2
mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.minor.width'] = 2
mpl.rcParams['ytick.major.width'] = 2

# use colorblind seaborn style
plt.style.use('seaborn-colorblind')

/home/alejandro/miniconda3/envs/py39/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2024-04-21 09:59:16.257328: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 09:59:25.835512: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/tmp/ipykernel_1144913/773372684.py:36: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt

# Full Data

In [3]:
adata_full = sc.read_h5ad('../integration/GBMOrganoids_scVIsurgery20240408.h5ad')

In [4]:
adata_full.obs.head()

,cell_id,x_centroid,y_centroid,transcript_counts,control_probe_counts,control_codeword_counts,unassigned_codeword_counts,deprecated_codeword_counts,total_counts,cell_area,...,leiden_1.1,leiden_1.2,leiden_1.3,leiden-bbknn_1.1,leiden-bbknn_1.05,leiden-bbknn_1.2,leiden-bbknn_1.15,leiden-bbknn_1.3,annotation_20240402,annotation_20240408
aaaocedi-1-0,aaaocedi-1,828.616821,720.303223,281,0,0,0,0,281,96.950472,...,8,8,8,3,8,11,4,11,+HR1,+HYP1
aadidjpi-1-0,aadidjpi-1,845.581909,721.379333,271,0,0,0,0,271,105.394691,...,4,3,11,3,5,11,6,11,+HR1,+HYP1
aaeaihbj-1-0,aaeaihbj-1,820.322021,724.660339,175,0,0,1,1,177,70.082503,...,8,8,8,11,8,12,6,12,+HR2,rAC
aaggikdb-1-0,aaggikdb-1,824.630554,730.077209,167,0,0,0,0,167,60.464221,...,4,3,4,3,0,11,4,11,+HR1,+HYP1
aagiopif-1-0,aagiopif-1,852.391968,735.970764,566,0,0,0,0,566,202.570945,...,4,3,4,3,0,2,4,3,+HYP2,+HYP1


In [5]:

exp = pd.Categorical([l+'-'+c+'-'+t for l, c, t in zip(adata_full.obs.line, adata_full.obs.condition, adata_full.obs.time)])
adata_full.obs['Experiment'] = exp
cell_n = {e: (adata_full.obs['Experiment']==e).sum() for e in adata_full.obs['Experiment'].cat.categories}
organoid_size = np.array([cell_n[e] for e in adata_full.obs['Experiment']])

# Find the center for each organoid
import shapely as shp
from scipy.spatial.distance import euclidean

centroid_organoid = {}
for exp in adata_full.obs['Experiment'].cat.categories:
    ad = adata_full[adata_full.obs.Experiment == exp]
    ps = np.array([ad.obs.x_centroid.values, ad.obs.y_centroid.values])
    mp = shp.MultiPoint(ps.T)
    centroid_organoid[exp] = np.array(mp.convex_hull.centroid.coords)
organoid_size = np.array([cell_n[e] for e in adata_full.obs['Experiment']])

adata_full.obs['dist2core'] = np.array([euclidean(np.array([x,y]), centroid_organoid[exp][0,:]) for x,y,e in zip(adata_full.obs.x_centroid, adata_full.obs.y_centroid, adata_full.obs.Experiment)])

In [6]:
test_hyp_vs_pla = pd.read_parquet('../differential_expression/DifferentialExpression_hyp_vs_pla.parquet')
test_hyp_vs_hyppla = pd.read_parquet('../differential_expression/DifferentialExpression_hyp_vs_hyppla.parquet')
test_pla_vs_hyppla = pd.read_parquet('../differential_expression/DifferentialExpression_pla_vs_hyppla.parquet')

In [7]:
genes_for_model = []
for test in [(test_hyp_vs_pla, .5), (test_hyp_vs_hyppla, 0.032), (test_pla_vs_hyppla, .1)]:
    df, val = test
    df = df[(df['log2fc'] >= val) | (df['log2fc'] <= -val)]
    genes_for_model += df.gene.tolist()


In [8]:
genes_for_model = np.unique(np.array(genes_for_model))

In [9]:
df = pd.DataFrame(data=adata_full.layers['counts'].toarray(), columns=adata_full.var_names)
df['Condition'] = adata_full.obs.condition.values


df['hyp'] = [1 if c == 'hyp' and t != '144h' else 0 for c,t in zip(adata_full.obs.condition, adata_full.obs.time)]
df['pla'] = [1 if c == 'pla' and t != '144h' else 0 for c,t in zip(adata_full.obs.condition, adata_full.obs.time)]
df['hyppla'] = [1 if c == 'hyppla' and t != '144h' else 0 for c,t in zip(adata_full.obs.condition, adata_full.obs.time)]


df['dist2core'] = adata_full.obs['dist2core'].values

time_dic = {'000h':0, '024h':1, '072h':2, '144h':3}
df['time'] = np.array([time_dic[t] for t in adata_full.obs.time.values])

df['SL007'] = np.array([1 if l == 'SL007' else 0for l in adata_full.obs.line])
df['SL013'] = np.array([1 if l == 'SL013' else 0for l in adata_full.obs.line])
df['SL014'] = np.array([1 if l == 'SL014' else 0for l in adata_full.obs.line])
df['SL027A'] = np.array([1 if l == 'SL27A' else 0for l in adata_full.obs.line])

In [ ]:
for gene in genes_for_model:
#for gene in ['VEGFA']:
    print(gene)
    #try:
    with pm.Model() as mdl_gbo:
        # define priors, weakly informative Normal   
        minv = df[gene].min()
        y0 = pm.Normal("Intercept", mu=minv, sigma=1)

        hyp = pm.Normal("hyp", mu=0, sigma=1.5)
        pla = pm.Normal("plasma", mu=0, sigma=1.5)
        hyppla = pm.Normal("hypoxia:plasma", mu=0, sigma=1.5)

        dist2core = pm.Normal("dist2core", mu=0, sigma=1)  
        #scale = pm.Exponential('scale', 1.)
        #t = pm.Normal("t", mu=0, sigma=10)

        # define linear model and exp link function
        theta = pm.math.exp(dist2core * df['dist2core']) * (
            y0 
            
            +

            (
                hyp * df["hyp"].values
                + pla * df["pla"].values
                + hyppla * df["hyppla"].values
            )

        )

        ## Define Poisson likelihood

        y = pm.Poisson("y", mu=pm.math.exp(theta), observed=df[gene].values)

    with mdl_gbo:
        inf_gbo = pm.sample(tune=2000,draws=500)
        # inf_gbo.extend(pm.sample_posterior_predictive(inf_gbo))

    inf_gbo.to_netcdf(f"traces/{gene}.nc",)

    az.plot_trace(inf_gbo,)
    plt.subplots_adjust(hspace=.4)
    plt.savefig(f"plots/{gene}.png")
#    except:
#        pass

VEGFA


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Intercept, hyp, plasma, hypoxia:plasma, dist2core]
